In [88]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [89]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [(sigma_plus**v[i][j] if v[i][j] == 1 else np.eye(2)) for j in range(n)])
        term2 = reduce(np.kron, [(sigma_minus**w[i][j] if w[i][j] == 1 else np.eye(2)) for j in range(n)])
        term3 = reduce(np.kron, [(sigma_plus**w[i][j] if w[i][j] == 1 else np.eye(2)) for j in range(n)])
        term4 = reduce(np.kron, [(sigma_minus**v[i][j] if v[i][j] == 1 else np.eye(2)) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr

In [101]:
v = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])
w = np.array([[0, 1, 0], 
              [0, 0, 1],
              [1, 0, 0]])

# v = np.array([[1, 0, 1], 
#               [0, 1, 0]])
# w = np.array([[0, 0, 0], 
#               [1, 0, 0]])
num = 3
# print("x1+x2-x3=c")
# for i in range(2**num):
#   B = np.zeros(2**num)
#   B[i] = 1
#   C = calculate_hamiltonian(v, w)
#   print(first_nonzero_index(B, num))
#   print(first_nonzero_index(C.dot(B), num))
#   print("====")

In [103]:
def gnrt_Hd():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += add_in_target(3, 0, GateZ)
  Hd += add_in_target(3, 1, GateZ)
  Hd += add_in_target(3, 2, GateZ)
  return Hd


In [102]:
C = calculate_hamiltonian(v, w)
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(C)
print(eigenvalues)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[-1.  2. -1.  2. -1. -1.  0.  0.]

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.4082', '0.0000', '0.4082', '-0.8165', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.0000', '0.0000', '0.5774', '0.0000', '0.5774', '0.5774', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.7682', '-0.3841', '-0.1384', '-0.3841', '-0.1384', '0.2768', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.5485', '-0.5485', '-0.1803', '-0.5485', '-0.1803', '-0.1803', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.4153', '0.3649', '-0.1835', '-0.7802', '-0.0410', '0.2244', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '-0.2422', '0.0380', '-0.6350', '0.2042', '0.7005', '-0.0654', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000']


In [104]:
C = gnrt_Hd()
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(C)
print(eigenvalues)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[ 3.+0.j  1.+0.j  1.+0.j -1.+0.j  1.+0.j -1.+0.j -1.+0.j -3.+0.j]

Eigenvalues:
3.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
1.0000
Eigenvectors:
['0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
1.0000
Eigenvectors:
['0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000']

Eigenvalues:
-3.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000']


In [94]:

num = 3
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = gnrt_Hd()
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

000
000
====
001
001
====
010
010
====
011
011
====
100
100
====
101
101
====
110
110
====
111
111
====


In [113]:
import numpy as np

Hd = calculate_hamiltonian(v, w)
C = gnrt_Hd()

AB = np.dot(Hd, C)
BA = np.dot(C, Hd)

# 验证是否对易
print("对易" if np.array_equal(AB, BA) else "不对易")

对易


In [116]:
# x1+x2+x3=c 的 Hd
eigenvalues, eigenvectors = np.linalg.eig(Hd)
print(eigenvalues)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[-1.  2. -1.  2. -1. -1.  0.  0.]

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.4082', '0.0000', '0.4082', '-0.8165', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.0000', '0.0000', '0.5774', '0.0000', '0.5774', '0.5774', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.7682', '-0.3841', '-0.1384', '-0.3841', '-0.1384', '0.2768', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.5485', '-0.5485', '-0.1803', '-0.5485', '-0.1803', '-0.1803', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.4153', '0.3649', '-0.1835', '-0.7802', '-0.0410', '0.2244', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '-0.2422', '0.0380', '-0.6350', '0.2042', '0.7005', '-0.0654', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000']


In [115]:
# ∑σz x1+x2+x3 的 C
eigenvalues, eigenvectors = np.linalg.eig(C)
print(eigenvalues.real)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[ 3.  1.  1. -1.  1. -1. -1. -3.]

Eigenvalues:
3.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
1.0000
Eigenvectors:
['0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
1.0000
Eigenvectors:
['0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000']

Eigenvalues:
-3.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000']
